In [23]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [29]:
data = pd.read_csv('./output.csv')
data = data.drop('text', axis=1)

In [30]:
data['label'] = data['label'].astype('category')
categories = dict(enumerate(data['label'].cat.categories))
data['label'] = data['label'].cat.codes

In [31]:
dep_one_hot = pd.get_dummies(data['dep'], prefix='dep')
pos_one_hot = pd.get_dummies(data['pos'], prefix='pos')
head_text_one_hot = pd.get_dummies(data['head_text'], prefix='head_text')
head_pos_one_hot = pd.get_dummies(data['head_pos'], prefix='head_pos')
head_dep_one_hot = pd.get_dummies(data['head_dep'], prefix='head_dep')
next_verb_one_hot = pd.get_dummies(data['next_verb'], prefix='next_verb')
entity_label_one_hot = pd.get_dummies(data['entity_label'], prefix='entity_label')

#drop categorical colums
data = data.drop(['dep', 'pos', 'head_text', 'head_pos', 'head_dep', 'next_verb', 'entity_label'], axis=1)
data = data.join(dep_one_hot)
data = data.join(pos_one_hot)
#data = data.join(head_text_one_hot)
data = data.join(head_pos_one_hot)
data = data.join(head_dep_one_hot)
#data = data.join(next_verb_one_hot)
data = data.join(entity_label_one_hot)

print(data.shape)

(4723, 1257)


In [32]:
X = data.drop('label', axis=1)
y = data['label']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [34]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [35]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8196443691786621
